In [1]:
import nltk
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [6]:
trainSet=pd.read_csv('trainNlp.csv')
testSet=pd.read_csv('testNlp.csv')
nltk.download('stopwords')

trainModify=trainSet
testModify=testSet

trainModify=replaceUrlandHash(trainModify)
testModify=replaceUrlandHash(testModify)

trainModify=removePunctuationAndStopWords(trainModify)
testModify=removePunctuationAndStopWords(testModify)

trainModify=sentenceStemmer(trainModify)
testModify=sentenceStemmer(testModify)


pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

pipeline.fit(trainModify['tweet'],trainModify['label'])

predictions = pipeline.predict(testModify['tweet'])

print(predictions)

testModify['label']=predictions

testModify[['id'],['label']].to_csv('SubmissionsNlp.csv',index=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sumanth/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


[1 1 1 ... 1 0 0]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [8]:



def replaceUrlandHash(requiredDataset):
    i=0
    for row in requiredDataset['tweet']:
        row=row.replace('#','')
        index=len(row)
        #print(row)
        if(row.find('http://')!=-1):
            index=row.find('http://')
        elif(row.find('https://')!=-1):
            index=row.find('https://')
        row=row[:index]
        requiredDataset['tweet'][i]=row
        i=i+1
    
    return  requiredDataset   
       

In [4]:
# Check characters to see if they are in punctuation

def removePunctuationAndStopWords(requiredDataset):
    i=0
    for row in requiredDataset['tweet']: 
        nopunc = [char for char in row if char not in string.punctuation]

        # Join the characters again to form the string.
        nopunc = ''.join(nopunc)

        # Now just remove any stopwords
        nonStopWords=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')] 

        nonStopWords=' '.join(nonStopWords)

        requiredDataset['tweet'][i]=nonStopWords

        i=i+1
        
    return requiredDataset        

In [5]:

def sentenceStemmer(requiredDataset):

    porter = PorterStemmer()
    nopunc=[]
    stemWords=[]
    i=0
    for row in requiredDataset['tweet']:
        token_words=word_tokenize(row)
        stem_sentence=[]
        for word in token_words:
            stem_sentence.append(porter.stem(word))
            stem_sentence.append(" ")
        stem_sentence="".join(stem_sentence) 

        requiredDataset['tweet'][i]=stem_sentence

        i=i+1 
        
    return requiredDataset   

[0 0 0 ... 0 0 0]


In [29]:
print (classification_report(trainSet['label'], all_predictions))

             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5894
          1       0.94      0.69      0.79      2026

avg / total       0.91      0.91      0.90      7920

